# Solution strategies
This tutorials demonstrates some of the more advanced solution strategies available in `PorePy`. 
The tutorial will be supplemented with more strategies as these become available.
The model problems are presented succinctly.

## Line search for multiphysics problems with fracture deformation (experimental)
The combination of highly nonlinear multiphysics problems and the non-smooth formulation of contact mechanics may lead to severe convergence issues.
To improve convergence, we provide a line search algorithm tailored to the irregularities of the contact equations, see https://arxiv.org/abs/2407.01184.


We define a dummy problem with one fracture and trivial problem data (boundary conditions etc.). 
This allows us to emphasize the steps needed to invoke the line search algorithm.
For more challenging problems which would not converge with regular Newton, please confer the examples of the above manuscript.

In [ ]:
import porepy as pp

from porepy.applications.md_grids.model_geometries import CubeDomainOrthogonalFractures
from porepy.numerics.nonlinear import line_search


class TailoredPoromechanics(
    # Problem definition
    CubeDomainOrthogonalFractures,
    # Add nonlinearity to the fracture flow
    pp.constitutive_laws.CubicLawPermeability,
    # Needed for the tailored line search algorithm
    pp.models.solution_strategy.ContactIndicators,
    # Base class
    pp.poromechanics.Poromechanics,
):
    """Combine mixins with the poromechanics class."""


To invoke the line search, we first define the tailored Newton solver class. We then specify a few parameters, including the "nonlinear_solver". 

In [3]:
class ConstraintLineSearchNonlinearSolver(
    line_search.ConstraintLineSearch,  # The tailoring to contact constraints.
    line_search.SplineInterpolationLineSearch,  # Technical implementation of the actual search along given update direction
    line_search.LineSearchNewtonSolver,  # General line search.
):
    """Collect all the line search methods in one class."""

model_params = {
    "fracture_indices": [1], # Fracture with constant y-coordinate
    "meshing_arguments": {"cell_size": 1 / 4},
}
solver_params = {
    "nl_convergence_tol_res": 1e-10, # Since the line search reduces the update, a residual based tolerance is needed
    "nonlinear_solver": ConstraintLineSearchNonlinearSolver,
    "Global_line_search": 0,  # Set to 1 to use turn on a residual-based line search
    "Local_line_search": 1,  # Set to 0 to use turn off the tailored line search
    "adaptive_indicator_scaling": 1,  # Scale the indicator adaptively to increase robustness
}
# Run the simulation.
model = TailoredPoromechanics(model_params)
pp.run_time_dependent_model(model, solver_params)

Some known issues to consider:
- The nonlinearity caused by the fracture permeability has proven sensitive to discretization method.
TPFA seems considerably more stable than MPFA.
For details, see the above mentioned paper.
- If regular Newton converges, it may be faster than the line search method.
An unexplored option is to use the line search as a fallback if Newton fails to converge.